In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
#实现多输出
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab  
import matplotlib.pyplot as plt  
import jieba
import textdistance
import random

In [2]:
data = pd.read_excel('cleaned_new.xlsx')

In [3]:
data.head()

,Unnamed: 0,PRIMARY_NAME,ALT_NAME,State,Cosine Similarity,Levenshtein Distance,Jaro-Winkler Distance,Jaccard Index,Monge Elkan,MRA,Longest Common Substring,Longest Common Subsequence,In String Search
0,0,gazprombank upravlenie aktivami,gazprombank asset management zao,Match,0.666751,15,0.847686,0.500000,0.024974,4,12,18,0
1,1,gazprombank upravlenie aktivami,closed joint-stock company gazprombank-upravle...,Match,0.731083,30,0.683537,0.534483,0.032258,3,19,30,0
2,2,united instrument manufacturing corporation,jsc-united-instrument-manufacturing-corporation,Match,0.889768,40,0.802099,0.800000,0.021633,3,13,40,0
3,3,gaz-oil ooo,gaz-oil,Match,0.763763,7,0.916667,0.583333,0.069444,4,7,7,1
4,4,ken wong,ken wong kaw,Match,0.816497,8,0.933333,0.666667,0.125000,3,8,8,1


In [4]:
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

In [5]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()

In [6]:
sample = random.sample(range(len(data)),10000)
name = data.loc[sample]['PRIMARY_NAME'].append(data.loc[sample]['ALT_NAME']).fillna('')
x = vectorizer.fit_transform(name)
tfidf = transformer.fit_transform(x)
word = vectorizer.get_feature_names()

In [7]:
weight = tfidf.toarray().astype(np.float16)

In [8]:
#降维
def PCA(weight, dimension):
 
    from sklearn.decomposition import PCA
 
    print('原有维度: ', len(weight[0]))
    print('开始降维:')
 
    pca = PCA(n_components=dimension) # 初始化PCA
    X = pca.fit_transform(weight) # 返回降维后的数据
    #cumsum=np.cumsum(pca.explained_variance_ratio_)
    #d=np.argmax(cumsum>=0.95)+1
    print('降维后维度: ', len(X[0]))
    print(X)
    #print(d)
    return X

In [9]:
a = PCA(weight, 20)

原有维度:  11454
开始降维:
降维后维度:  20
[[ 0.03941327 -0.07441442 -0.03784762 ...  0.00301714 -0.03683408
  -0.00254237]
 [ 0.03513255 -0.08755514 -0.06031842 ...  0.01007033 -0.00994188
   0.00118153]
 [-0.15502856  0.09635905 -0.10159186 ...  0.17334391  0.07770223
  -0.07519154]
 ...
 [ 0.01052455 -0.02050255 -0.01003111 ... -0.00241116  0.01184002
  -0.01080792]
 [ 0.03902981 -0.07603654 -0.02673306 ...  0.0061424  -0.03769171
  -0.00297643]
 [ 0.03508547 -0.06806523 -0.03436891 ...  0.00688829 -0.03398456
   0.00244527]]
1


In [26]:
len(a[0])

20

In [10]:
#k均值聚类

def kmeans(X, k): # X=weight
 
    from sklearn.cluster import KMeans
 
    print('开始聚类:')
 
    clusterer = KMeans(n_clusters=k, init='k-means++') # 设置聚类模型
 
    # X = clusterer.fit(weight) # 根据文本向量fit
    # print X
    # print clf.cluster_centers_
 
    # 每个样本所属的簇
    y = clusterer.fit_predict(X) # 把weight矩阵扔进去fit一下,输出label
    print(y)
 
    # i = 1
    # while i <= len(y):
    #     i += 1
 
    # 用来评估簇的个数是否合适,距离约小说明簇分得越好,选取临界点的簇的个数
    # print clf.inertia_
 
    return y



In [42]:
k = 4
from sklearn.cluster import KMeans
s = KMeans(n_clusters=k, init='k-means++').fit(a)


In [11]:
def Silhouette(X, y):
 
    from sklearn.metrics import silhouette_samples, silhouette_score
 
    print ('计算轮廓系数:')
 
    silhouette_avg = silhouette_score(X, y) # 平均轮廓系数
    sample_silhouette_values = silhouette_samples(X, y) # 每个点的轮廓系数
 
    print(silhouette_avg)
 
    return silhouette_avg, sample_silhouette_values


In [ ]:
#######################################

In [48]:
from sklearn.decomposition import PCA
dimension = 20
pca = PCA(n_components=dimension)

In [50]:
pca.fit(weight)

PCA(n_components=20)

In [53]:
pca.transform([[i for i in range(11454)]])

array([[-6238.16403621,  1168.789169  , -3054.88141548,   274.87603038,
        -4432.64057181,  1804.59406289, -5631.77615581, -3267.14209136,
        -2944.45214171, -3911.510614  ,  1965.23708784, -4690.89432381,
         4366.49823631, -4723.50996508,   -25.49855689,  4470.48772553,
         2943.78088396,  1356.08949865,   852.24149195,  4400.83730971]])

In [ ]:
#################################################

In [12]:
k = 4
x = kmeans(a, k)

开始聚类:
[1 1 3 ... 1 1 1]


In [13]:
X = a
silhouette_avg1, sample_silhouette_values1 = Silhouette(a, x)

计算轮廓系数:
0.31711953187033315


In [55]:
data1 = data.iloc[sample,:]
data1['class_1'] = x[:10000]
data1['class_2'] = x[10000:]

<ipython-input-55-739a993bd10f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['class_1'] = x[:10000]
<ipython-input-55-739a993bd10f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['class_2'] = x[10000:]


In [56]:
index1 = data1[data1['State'] == 'Match']['State'].index
index2 = data1[data1['class_1'] == data1['class_2']]['State'].index
len(index1) #总的match数量
len(index2)/10000  #聚类之后同一行是一类的数量
sum([i in index2 for i in index1]) #聚类之后同一行是一类且match的数量

129

0.6035

119

In [16]:
#可以看出只有六个match在聚类之后认为不是同一类。但是因为聚类只能进行抽样，最后用于训练的数据数量大大减少，使得模型的效果不如直接训练。

In [23]:
len(weight[0])

11454

In [22]:
[i for i in weight[0] if i!=0]

[0.506, 0.795, 0.335]